# SCohenLab 2D BATCH Image Processing notebook (Simplified MCZ)

--------------
# PIPELINE OVERVIEW
## ❶ GOAL SETTING ✍

### GOAL:  Infer sub-cellular components in order to understand interactome 

To measure shape, position, size, and interaction of eight organelles/cellular components (Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and SOMA) during differentiation of iPSCs, in order to understand the Interactome / Spatiotemporal coordination.

### summary of _OBJECTIVES_ ✅
- robust inference of subcellular objects:
  + 1️⃣-***nuclei***
  + 2️⃣-***soma***
  + 3️⃣-***cytosol*** (+ ***nucleus***)
  + 4️⃣-***lysosome***
  + 5️⃣-***mitochondria***
  + 6️⃣-***golgi***
  + 7️⃣-***peroxisome***
  + 8️⃣-***endoplasmic reticulum***
  + 9️⃣-***lipid body***





## ❷ DATA CREATION
> METHODS:📚📚
> 
> iPSC lines prepared and visualized on Zeiss Microscopes. 32 channel multispectral images collected.  Linear Unmixing in  ZEN Blue software with target emission spectra yields 8 channel image outputs.  Channels correspond to: Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and a “residual” signal.

> Meta-DATA 🏺 (artifacts)
>   - Microcope settings
>  - OME scheme
> - Experimenter observations
> - Sample, bio-replicate, image numbers, condition values, etc
>  - Dates
>  - File structure, naming conventions
>  - etc.





## ❸. IMAGE PROCESSING ⚙️🩻🔬
### INFERENCE OF SUB-CELLULAR OBJECTS
The imported images have already been pre-processed to transform the 32 channel spectral measuremnts into "linearly unmixed" images which estimate independently labeled sub-cellular components.  Thes 7 channels (plus a residual "non-linear" signal) will be used to infer the shapes and extents of these sub-cellular components.   
We will perform computational image analysis on the pictures (in 2D an 3D) to _segment_ the components of interest for measurement.  In other prcoedures we can used these labels as "ground truth" labels to train machine learning models to automatically perform the inference of these objects.
Pseudo-independent processing of the imported multi-channel image to acheive each of the 9 objecives stated above.  i.e. infering: NUCLEI, SOMA, CYTOSOL, LYSOSOME, MITOCHONDRIA, GOLGI COMPLEX, PEROZISOMES, ENDOPLASMIC RETICULUM, and LIPID BODIES

### General flow for infering objects via segmentation
- Pre-processing 🌒
- Core-processing (thresholding) 🌕
- Post-processing  🌘

### QC 🚧 WIP 🚧 




## ❹. QUANTIFICATION 📏📐🧮

SUBCELLULAR COMPONENT METRICS
-  extent 
-  size
-  shape
-  position



### NOTE: PIPELINE TOOL AND DESIGN CHOICES?
We want to leverage the Allen Cell & Structure Setmenter.  It has been wrapped as a [napari-plugin](https://www.napari-hub.org/plugins/napari-allencell-segmenter) but fore the workflow we are proving out here we will want to call the `aicssegmentation` [package](https://github.com/AllenCell/aics-segmentation) directly.

#### ​The Allen Cell & Structure Segmenter 
​The Allen Cell & Structure Segmenter is a Python-based open source toolkit developed at the Allen Institute for Cell Science for 3D segmentation of intracellular structures in fluorescence microscope images. This toolkit brings together classic image segmentation and iterative deep learning workflows first to generate initial high-quality 3D intracellular structure segmentations and then to easily curate these results to generate the ground truths for building robust and accurate deep learning models. The toolkit takes advantage of the high replicate 3D live cell image data collected at the Allen Institute for Cell Science of over 30 endogenous fluorescently tagged human induced pluripotent stem cell (hiPSC) lines. Each cell line represents a different intracellular structure with one or more distinct localization patterns within undifferentiated hiPS cells and hiPSC-derived cardiomyocytes.

More details about Segmenter can be found at https://allencell.org/segmenter
In order to leverage the A




## IMPORTS

In [1]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict
from typing import Optional, Union

import numpy as np
import pandas as pd
from skimage.measure import regionprops_table, regionprops

from aicssegmentation.core.pre_processing_utils import  image_smoothing_gaussian_slice_by_slice 
from skimage.measure import label

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.utils.file_io import (read_czi_image,
                                                                    export_inferred_organelle,
                                                                    list_image_files)

from infer_subc_2d.utils.img import *
from infer_subc_2d.organelles import * 

%load_ext autoreload
%autoreload 2



In [2]:
# NOTE:  these "constants" are only accurate for the testing MCZ dataset
from infer_subc_2d.constants import (TEST_IMG_N,
                                                                    NUC_CH ,
                                                                    LYSO_CH ,
                                                                    MITO_CH ,
                                                                    GOLGI_CH ,
                                                                    PEROXI_CH ,
                                                                    ER_CH ,
                                                                    LIPID_CH ,
                                                                    RESIDUAL_CH )              


## SETUP

In [3]:
# this will be the example image for testing the pipeline below
test_img_n = TEST_IMG_N

# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
in_data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(in_data_path,im_type)
test_img_name = img_file_list[test_img_n]

# save output ".tiff" files here
out_data_path = data_root_path / "out"

if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [4]:
img_data,meta_dict = read_czi_image(test_img_name)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [5]:
scale[0]/scale[1], scale

(7.267318290023735,
 (0.5804527163320905, 0.07987165184837318, 0.07987165184837318))

Now get the single "optimal" slice of all our organelle channels....

## get the inferred soma, nuclei and cytosol objects

(takes < 1 sec)

Builde the segmentations in order




In [6]:
###################
# SOMA, NUCLEI, CYTOSOL, NUCLEUS
###################
nuclei_obj =  get_nuclei(img_data,meta_dict, out_data_path)
soma_obj  = get_soma(img_data, nuclei,meta_dict, out_data_path)
cytosol_mask = get_cytosol(nuclei_obj , soma_obj , meta_dict, out_data_path)


loaded nuclei from /Users/ahenrie/Projects/Imaging/data/out


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


loaded soma from /Users/ahenrie/Projects/Imaging/data/out


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


loaded cytosol from /Users/ahenrie/Projects/Imaging/data/out


In [7]:

###################
# LYSOSOME
###################
lysosome_obj  = get_lysosome(img_data,meta_dict, out_data_path)

###################
# MITOCONDRIA
###################
mitochondria_obj  = get_mitochondria(img_data,meta_dict, out_data_path)
###################
#  GOLGI
###################
golgi_obj = get_golgi(img_data,meta_dict, out_data_path)

###################
#  PEROXISOME
###################
peroxisome_obj  = get_peroxisome(img_data,meta_dict, out_data_path)

###################
#  ER
###################
er_obj  = get_endoplasmic_reticulum(img_data,meta_dict, out_data_path)

###################
#  LIPID BODIES
###################
lipid_obj  =  get_lipid(img_data,meta_dict, out_data_path)

/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


loaded lysosome from /Users/ahenrie/Projects/Imaging/data/out


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


loaded mitochondria from /Users/ahenrie/Projects/Imaging/data/out


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


...

loaded peroxisome from /Users/ahenrie/Projects/Imaging/data/out


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


loaded endoplasmic reticulum from /Users/ahenrie/Projects/Imaging/data/out


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


loaded lipid from /Users/ahenrie/Projects/Imaging/data/out


-------------------------
## regionprops

`skimage.measure.regionprops` provides the basic tools nescessary to quantify our segmentations.

First lets see what works in 3D.  

> Note: the names of the regionprops correspond to the 2D analysis even for those which are well defined in 3D.  i.e. "area" is actually "volume" in 3D, etc.

In [14]:
lysosome_obj.shape

(16, 768, 768)

In [17]:
labels = label(nuclei_obj )
rp = regionprops(labels, intensity_image=img_data[NUC_CH])

supported = [] 
unsupported = []

for prop in rp[0]:
    try:
        rp[0][prop]
        supported.append(prop)
    except NotImplementedError:
        unsupported.append(prop)

print("Supported properties:")
print("  " + "\n  ".join(supported))
print()
print("Unsupported properties:")
print("  " + "\n  ".join(unsupported))

Supported properties:
  area
  bbox
  bbox_area
  centroid
  convex_area
  convex_image
  coords
  equivalent_diameter
  euler_number
  extent
  feret_diameter_max
  filled_area
  filled_image
  image
  inertia_tensor
  inertia_tensor_eigvals
  intensity_image
  label
  local_centroid
  major_axis_length
  max_intensity
  mean_intensity
  min_intensity
  minor_axis_length
  moments
  moments_central
  moments_normalized
  slice
  solidity
  weighted_centroid
  weighted_local_centroid
  weighted_moments
  weighted_moments_central
  weighted_moments_normalized

Unsupported properties:
  eccentricity
  moments_hu
  orientation
  perimeter
  perimeter_crofton
  weighted_moments_hu


In [9]:
viewer = napari.view_image(rp[0].convex_image)



TypeError: 'function' object is not subscriptable

In [53]:

def get_summary_stats_3D(input_obj, intensity_img, mask):
    """ collect volumentric stats """

    # mask
    # intensity_img = apply_mask(intensity_img,mask )  #not needed
    input_obj = apply_mask(input_obj,mask ) 
    labels = label(input_obj ).astype("int")
    properties = ["label"]
    # add intensity:
    properties = properties + ['max_intensity', 'mean_intensity', 'min_intensity']
    # arguments must be in the specified order, matching regionprops
    def standard_deviation_intensity(region, intensities):
        return np.std(intensities[region])
    extra_properties = [standard_deviation_intensity]

    # add area
    properties = properties + ['area', 'equivalent_diameter'] 
    #  position:
    properties = properties + ['centroid', 'bbox'] #, 'bbox', 'weighted_centroid']
    # etc
    properties = properties +  [ 'euler_number', 'extent'] # only works for BIG organelles: 'convex_area','solidity',

    props = regionprops_table(labels, intensity_image=intensity_img,
                                properties=properties, extra_properties=extra_properties)
    
    props["surface_area"] = surface_area_from_props(labels,props)

    stats_table = pd.DataFrame(props)
    stats_table.rename({"area": "volume"})
    
    #  # ETC.  skeletonize via cellprofiler /Users/ahenrie/Projects/Imaging/CellProfiler/cellprofiler/modules/morphologicalskeleton.py
    #         if x.volumetric:
    #             y_data = skimage.morphology.skeletonize_3d(x_data)
    # /Users/ahenrie/Projects/Imaging/CellProfiler/cellprofiler/modules/measureobjectskeleton.py

    rp = regionprops(labels, intensity_image=intensity_img, extra_properties=extra_properties)
    return stats_table, rp

In [39]:
from scipy.ndimage import find_objects
    
labels = label(nuclei_obj ).astype("int")
objects = find_objects(labels)

labels[objects[3]]

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [40]:
from skimage.measure import mesh_surface_area, marching_cubes

def surface_area_from_props(labels,props):
    # SurfaceArea
    surface_areas = np.zeros(len(props["label"]))
    #spacing = [1, 1, 1] # this is where we could deal with anisotropy
    
    for index, lab in enumerate(props["label"]):
        # this seems less elegant than you might wish, given that regionprops returns a slice,
        # but we need to expand the slice out by one voxel in each direction, or surface area freaks out
        volume= labels[max(props['bbox-0'][index]-1,0):min(props['bbox-3'][index]+1,labels.shape[0]),
                    max(props['bbox-1'][index]-1,0):min(props['bbox-4'][index]+1,labels.shape[1]),
                    max(props['bbox-2'][index]-1,0):min(props['bbox-5'][index]+1,labels.shape[2])] 
        volume = volume == lab
        verts, faces, _normals, _values = marching_cubes(
            volume,
            method="lewiner",
            spacing= (1.0,) * labels.ndim,
            level=0,
        )
        surface_areas[index] = mesh_surface_area(verts, faces)
        
    return surface_areas            


In [41]:
nuclei_table,nt = get_summary_stats_3D(nuclei_obj  , img_data[NUC_CH] ,soma_obj)


In [42]:
nuclei_table

,label,max_intensity,mean_intensity,min_intensity,area,equivalent_diameter,centroid-0,centroid-1,centroid-2,bbox-0,...,bbox-2,bbox-3,bbox-4,bbox-5,convex_area,euler_number,solidity,extent,standard_deviation_intensity,surface_area
0,1,41798,13653.069934,0,358522,88.139868,4.534673,363.840955,347.942068,0,...,217,12,476,465,409952,1,0.874546,0.535427,4894.172264,55770.65625


In [43]:


def get_summary_stats_2D(input_obj, intensity_img, mask):
    """ collect volumentric stats """

    # mask
    # intensity_img = apply_mask(intensity_img,mask )  #not needed
    input_obj = apply_mask(input_obj,mask ) 
    labels = label(input_obj ).astype("int")
    properties = ["label"]
    # add intensity:
    properties = properties + ['max_intensity', 'mean_intensity', 'min_intensity']
    # arguments must be in the specified order, matching regionprops
    def standard_deviation_intensity(region, intensities):
        return np.std(intensities[region])
    extra_properties = [standard_deviation_intensity]

    # add area
    properties = properties + ['area', 'equivalent_diameter'] 
    #  position:
    properties = properties + ['centroid', 'bbox'] 

    #  perimeter:
    properties = properties + ['perimeter', 'perimeter_crofton'] 

    table = regionprops_table(labels, intensity_image=intensity_img,
                                properties=properties, extra_properties=extra_properties)
    stats_table = pd.DataFrame(table)

    #  # ETC.  skeletonize via cellprofiler /Users/ahenrie/Projects/Imaging/CellProfiler/cellprofiler/modules/morphologicalskeleton.py
    #             y_data = skimage.morphology.skeletonize(x_data)
    # /Users/ahenrie/Projects/Imaging/CellProfiler/cellprofiler/modules/measureobjectskeleton.py

    rp = regionprops(labels, intensity_image=intensity_img, extra_properties=extra_properties)
    return stats_table, table, rp

In [82]:

def get_AintB_stats_3D(input_obj,mask):
    """ collect volumentric stats of A intersect B """

    # mask
    # intensity_img = apply_mask(intensity_img,mask )  #not needed
    input_obj = apply_mask(input_obj,mask ) 
    labels = label(input_obj ).astype("int")
    properties = ["label"]

    # add area
    properties = properties + ['area', 'equivalent_diameter'] 
    #  position:
    properties = properties + ['centroid', 'bbox'] #, 'bbox', 'weighted_centroid']
    # etc
    properties = properties +  [ 'euler_number', 'extent'] # only works for BIG organelles: 'convex_area','solidity',

    props = regionprops_table(labels, intensity_image=None,
                                properties=properties, extra_properties=[])
    
    props["surface_area"] = surface_area_from_props(labels,props)

    stats_table = pd.DataFrame(props)
    stats_table.rename({"area": "volume"})
    return stats_table

In [ ]:
# get overall summary stats for soma
florescence =  raw_soma_MCZ(img_data, scale_min_max=False)


In [ ]:
soma_table, st = get_summary_stats_3D(soma_obj, florescence,soma_obj)

soma_table

,label,max_intensity,mean_intensity,min_intensity,area,equivalent_diameter,centroid-0,centroid-1,centroid-2,bbox-0,...,bbox-3,bbox-4,bbox-5,convex_area,coords,euler_number,solidity,extent,standard_deviation_intensity,surface_area
0,1,397989,18123.944078,0,1493502,141.819428,5.477527,377.567316,298.462994,0,...,12,626,636,2086430,"[[0, 221, 478], [0, 221, 479], [0, 221, 480], ...",1,0.715817,0.401904,27336.770848,247666.953125


In [45]:

nuclei_table,nt = get_summary_stats_3D(nuclei_obj  , img_data[NUC_CH] ,soma_obj)

nuclei_table

,label,max_intensity,mean_intensity,min_intensity,area,equivalent_diameter,centroid-0,centroid-1,centroid-2,bbox-0,...,bbox-2,bbox-3,bbox-4,bbox-5,convex_area,euler_number,solidity,extent,standard_deviation_intensity,surface_area
0,1,41798,13653.069934,0,358522,88.139868,4.534673,363.840955,347.942068,0,...,217,12,476,465,409952,1,0.874546,0.535427,4894.172264,55770.65625


In [46]:
###################
get_methods  = [get_lysosome,
            get_mitochondria,
            get_golgi,
            get_peroxisome,
            get_endoplasmic_reticulum,
            get_lipid]

# load all the organelle segmentations
organelles = [meth(img_data,meta_dict, out_data_path) for meth in get_methods]



/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


loaded lysosome from /Users/ahenrie/Projects/Imaging/data/out


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


loaded mitochondria from /Users/ahenrie/Projects/Imaging/data/out


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


...

loaded peroxisome from /Users/ahenrie/Projects/Imaging/data/out


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


loaded endoplasmic reticulum from /Users/ahenrie/Projects/Imaging/data/out


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


loaded lipid from /Users/ahenrie/Projects/Imaging/data/out


In [96]:

# organelles = [nuclei_obj,
#                             lysosome_obj,
#                             mitochondria,
#                             golgi,
#                             peroxisome,
#                             er,
#                             lipid]

organelle_names = ["lysosome", "mitochondria","golgi","peroxisome","er","lipid"]
organelle_channels = [LYSO_CH,MITO_CH,GOLGI_CH,PEROXI_CH,ER_CH,LIPID_CH]

stats_tables = []
props = []
for org, ch in zip(organelles, organelle_channel):
    stats_tab, prps = get_summary_stats_3D(org , img_data[ch] ,cytosol_mask )
    stats_tables.append(stats_tab)
    props.append(prps)


In [104]:
ch_dict = dict(zip(organelle_names, organelle_channels))
idx_ = dict(zip(organelle_names, range(len(organelle_names) )) )

stats_tables[idx_["golgi"]]

,label,max_intensity,mean_intensity,min_intensity,area,equivalent_diameter,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,euler_number,extent,standard_deviation_intensity,surface_area
0,1,43596,20733.350410,2444,488,9.768032,5.450820,313.653689,196.518443,3,287,187,9,331,204,1,0.108734,7670.360227,977.503601
1,2,40702,16932.504225,1297,710,11.068444,7.316901,308.945070,223.019718,4,286,212,11,323,238,1,0.105435,6866.417188,1214.371216
2,3,65535,20435.236246,6,309,8.387901,6.514563,273.446602,213.462783,5,267,202,9,283,223,1,0.229911,13435.619055,563.978882
3,4,38312,18405.400000,1902,180,7.005266,7.483333,328.055556,248.955556,6,320,242,10,336,262,0,0.140625,9025.196513,403.187897
4,5,38969,21935.771930,6222,57,4.774839,7.561404,352.245614,212.052632,7,349,208,9,356,217,1,0.452381,8057.010399,137.336334
5,6,35850,14541.452991,4437,351,8.751910,8.547009,435.387464,261.424501,7,419,255,11,453,273,1,0.143382,5671.274279,719.153076
6,7,41541,20187.316583,5911,199,7.243550,8.713568,454.115578,304.748744,7,446,300,11,464,311,1,0.251263,6944.012459,410.460022
7,8,44158,16284.683849,0,1164,13.051230,9.656357,342.951031,301.720790,8,315,267,11,376,332,1,0.097856,6453.030865,2467.397217
8,9,47473,16424.094134,1816,733,11.186695,9.050477,355.259209,239.653479,8,331,227,11,382,255,1,0.171102,7118.680305,1295.627441
9,10,23837,13867.209091,5167,110,5.944719,8.490909,413.381818,229.481818,8,404,226,10,424,235,1,0.305556,3929.797841,289.394531


In [105]:

target = "lysosome"
organelle_names.index("lysosome")
target_i = organelle_names.index(target)
A = organelles[target_i ]
A.shape

0

In [92]:



for i,o in enumerate(organelle_names):
    

    if  i  != target_i:
        
        # get overall stats of intersection
        B = organelles[i]

        AintB = np.logical_and(A,B)
        _stats_tab = get_AintB_stats_3D(AintB, cytosol_mask)
    
        csv_path = out_data_path / f"{target}X{o}_{meta_dict['file_name'].split('/')[-1].split('.')[0]}_stats.csv"
        _stats_tab.to_csv(csv_path )

        break

    


2241

In [ ]:

for i,o in enumerate(organelle_names):
    if  i  != organelle_names.index(target):
        


        # get overall stats
        # overlap

        intersectA_B
        _stats_tab, rp = get_summary_stats_3D(org, img_data[ch], cytosol_mask)

        # 


        print(f"{o}- {organelle_channel[i]}")
        org = organelles[i]
        ch = organelle_channel[i]
        _stats_tab, rp = get_summary_stats_3D(org, img_data[ch], cytosol_mask)
        stats_bag.append(_stats_tab)

        csv_path = out_data_path / f"{o}_{meta_dict['file_name'].split('/')[-1].split('.')[0]}_stats.csv"
        
        _stats_tab.to_csv(csv_path )

        
        # get lablels of masked organelle

        # export
        break

In [55]:
org.shape

(16, 768, 768)

In [59]:
# def regionprops(label_image, intensity_image=None, cache=True,
#                 coordinates=None, *, extra_properties=None):
    
for j,org in enumerate(organelle_names):
    labels = label( organelles[j] ).astype("int")

    for i,lab in enumerate(_stats_tab['label']):
        idx = tuple(rp[i].coords.T)
        #target = org[idx]
        cmp_org = labels[idx]

        #total number of overlapping pixels
        overlap = sum(cmp_org>0)
        # overlap?

        org_js = np.unique(cmp_org)


        break
    break

touch_list

'lysosome{31}'

In [60]:
cmp_org[0]=2
org_js = np.unique(cmp_org)
org_js

array([ 2, 31])

In [38]:
# csv_path = out_data_path / f"{o}_{meta_dict["file_name"].split('/')[-1].split('.')[0]}_stats.csv"
meta_dict['file_name'].split('/')[-1].split('.')[0]

'ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed'

In [ ]:
###################
# MITOCONDRIA
###################
mito_obj  = get_mitochondria(img_data,meta_dict, out_data_path)
mito_table = get_summary_stats_3D( mito_obj, img_data[MITO_CH],cytosol_mask)

###################
#  GOLGI
###################
golgi_obj = get_golgi(img_data,meta_dict, out_data_path)
golgi_obj = get_summary_stats_3D( golgi_obj, img_data[GOLGI_CH],cytosol_mask)

###################
#  PEROXISOME
###################
peroxisome_obj  = get_peroxisome(img_data,meta_dict, out_data_path)

###################
#  ER
###################
er_obj  = get_endoplasmic_reticulum(img_data,meta_dict, out_data_path)

###################
#  LIPID BODIES
###################
lipid_obj  =  get_lipid(img_data,meta_dict, out_data_path)

In [36]:

viewer = napari.view_image(lysosome_obj)

In [37]:
viewer.add_image(florescence)
viewer.add_image(cytosol_mask)


<Image layer 'cytosol_mask' at 0x14a402a90>

In [20]:
from math import pi as PI

table['equivalent_diameter'], table['area'], (2 * 3 *  table['area'] / PI) ** (1 /3)


(array([141.81942778]), array([1493502]), array([141.81942778]))

In [40]:
###################
# SOMA, NUCLEI, CYTOSOL, NUCLEUS
###################
nuclei_obj =  infer_and_export_nuclei(img_data,meta_dict, out_data_path)

soma_obj = infer_and_export_soma(img_data, nuclei,meta_dict, out_data_path)
cytosol_mask =  infer_and_export_cytosol(soma_obj, nuclei_obj, meta_dict, out_data_path)



/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


saved file: /Users/ahenrie/Projects/Imaging/data/out/nuclei_ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.ome.tiff
inferred nuclei. wrote /Users/ahenrie/Projects/Imaging/data/out/nuclei_ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.ome.tiff
saved file: /Users/ahenrie/Projects/Imaging/data/out/soma_ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.ome.tiff
inferred soma. wrote /Users/ahenrie/Projects/Imaging/data/out/soma_ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.ome.tiff
saved file: /Users/ahenrie/Projects/Imaging/data/out/cytosol_ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.ome.tiff
inferred cytosol. wrote /Users/ahenrie/Projects/Imaging/data/out/cytosol_ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.ome.tiff


In [41]:
viewer.add_image(cytosol_mask)


<Image layer 'cytosol_mask [1]' at 0x167bf1490>

In [ ]:
###################
# LYSOSOME
###################
lysosome_obj  = get_lysosome(img_data,meta_dict, out_data_path)
florescence = apply_mask(img_data[LYSO_CH],cytosol_mask )  
lyso_table = get_summary_stats_3D(lysosome_obj, florescence)


In [ ]:

###################
# LYSOSOME
###################
lysosome = infer_and_export_lysosome(img_data,meta_dict, out_data_path)

###################
# MITOCONDRIA
###################
mitochondria = infer_and_export_mitochondria(img_data,meta_dict, out_data_path)
###################
#  GOLGI
###################
golgi = infer_and_export_golgi(img_data,meta_dict, out_data_path)

###################
#  PEROXISOME
###################
peroxisome = infer_and_export_peroxisome(img_data,meta_dict, out_data_path)

###################
#  ER
###################
er = infer_and_export_endoplasmic_reticulum(img_data,meta_dict, out_data_path)

###################
#  LIPID BODIES
###################
lipid =  infer_and_export_lipid(img_data,meta_dict, out_data_path)

In [ ]:
lipid

In [ ]:
from skimage.measure import regionprops_table

def collect_organelle_stats(
                                    img_in:np.ndarray, 
                                    florescence:Union[np.ndarray,None]=None
                                    ): 

    properties = ['label']
    extra_properties = []
    size = False 
    perimeter = False 
    shape = True 
    position = True 
    moments = True

    labels = label(img_in).astype("int").squeeze()
    intensity_image = None if florescence is None else florescence.squeeze()
    intensity = intensity_image is not None # depricate for now because the "infered object" doesn't have intensity


    if size:
        properties = properties + ['area', 'bbox_area',  'equivalent_diameter'] #'convex_area',

    if intensity:
        properties = properties + ['max_intensity', 'mean_intensity', 'min_intensity']
        # arguments must be in the specified order, matching regionprops
        def standard_deviation_intensity(region, intensities):
            return np.std(intensities[region])
        extra_properties.append(standard_deviation_intensity)

    if perimeter:
        if len(labels.shape) == 2:
            properties = properties + ['perimeter', 'perimeter_crofton']
        else:
            print("Perimeter measurements are not supported in 3D")
            # warnings.warn("Perimeter measurements are not supported in 3D")

    if shape:
        properties = properties + ['solidity', 'extent', 'feret_diameter_max', 'local_centroid']
        if len(labels.shape) == 2:
            properties = properties + ['major_axis_length', 'minor_axis_length', 'orientation', 'eccentricity']
            # we need these two to compute some shape descriptors
            if not size:
                properties = properties + ['area']
            if not perimeter:
                properties = properties + ['perimeter']
        else:
            properties = properties + ['moments_central']
        # euler_number,

    if position:
        properties = properties + ['centroid', 'bbox', 'weighted_centroid']

    if moments:
        properties = properties + ['moments', 'moments_normalized']
        if 'moments_central' not in properties:
            properties = properties + ['moments_central']
        if len(labels.shape) == 2:
            properties = properties + ['moments_hu']

    # todo:
    # weighted_local_centroid
    # weighted_moments
    # weighted_moments_central
    # weighted_moments_hu
    # weighted_moments_normalized

    # quantitative analysis using scikit-image's regionprops
    print(labels.shape)
    print(properties)
    table = regionprops_table(labels, intensity_image=intensity_image,
                                properties=properties, extra_properties=extra_properties)
    return table


In [ ]:




organelles = [nuclei,
                            lysosome,
                            mitochondria,
                            golgi,
                            peroxisome,
                            er,
                            lipid]

organelle_names = ["nuclei","lysosome", "mitochondria","golgi","peroxisome","er","lipid"]
organelle_channel = [NUC_CH,LYSO_CH,MITO_CH,GOLGI_CH,PEROXI_CH,ER_CH,LIPID_CH]

stats_bag = []
for i,o in enumerate(organelle_names):
    print(f"{o}- {organelle_channel[i]}")
    org = organelles[i]
    ch = organelle_channel[i]
    florescence = img_data[ch]

    # apply mask 
    mask = soma if ch==NUC_CH else cytosol

    org = apply_mask(org, mask)
    florescence = apply_mask(org,mask)  
    _stats = collect_organelle_stats(org, florescence)
    stats_bag.append(_stats)

    # get lablels of masked organelle

    # export



In [ ]:
stats_bag = []
for i,o in enumerate(organelle_names):
    print(f"{o}- {organelle_channel[i]}")
    org = organelles[i]
    ch = organelle_channel[i]
    florescence = img_data[ch]

    # apply mask 
    mask = soma if ch==NUC_CH else cytosol

    org = apply_mask(org, mask)
    florescence = apply_mask(org,mask)  
    _stats = collect_organelle_stats(org, florescence)
    stats_bag.append(_stats)

    # get lablels of masked organelle

    # export


    

In [ ]:

###################
# LYSOSOME
###################
lysosome_object =  fixed_infer_lysosome(img_data) 

###################
# MITOCONDRIA
###################
mito_object =  fixed_infer_mitochondria(img_data) 
###################
#  GOLGI
###################
golgi_object =  fixed_infer_golgi(img_data) 

###################
#  PEROXISOME
###################
peroxi_object =  fixed_infer_peroxisome(img_data) 

###################
#  ER
###################
er_object =  fixed_infer_endoplasmic_reticulum(img_data) 

###################
#  LIPID BODIES
###################
lipid_object =  fixed_infer_lipid(img_data) 


In [ ]:
img_layers = [soma_mask,
                            nuclei_object,
                            cytosol_mask,
                            lysosome_object,
                            mito_object,
                            golgi_object,
                            peroxi_object,
                            er_object,
                            lipid_object ]


organelle_names = ["soma","nuclei","cytosol","lysosome", "mitochondria","golgi","peroxisome"]

In [ ]:
labels = label(soma_mask).astype("int")


In [ ]:
regionprops_table(labels)

In [ ]:
# import pandas as pd
# # load region properties from csv file
# reg_props = pd.read_csv(csv_filename)
# try:
#     edited_reg_props = reg_props.drop(["Unnamed: 0"], axis=1)
# except KeyError:
#     edited_reg_props = reg_props

# if "label" not in edited_reg_props.keys().tolist():
#     label_column = pd.DataFrame(
#         {"label": np.array(range(1, (len(edited_reg_props) + 1)))}
#     )
#     edited_reg_props = pd.concat([label_column, edited_reg_props], axis=1)

In [ ]:
def handle_organelles(
        segmentation:Union[np.ndarray,None]=None, ,
        florescence:Union[np.ndarray,None]=None, 
        organelle: int,
        mask:Union[np.ndarray,None]=None, 
        # infer_params: Union[dict, None]=None,
        # prior:Union[Any,None]=None, 
        # export_nm:Union[Path, str,None]=None, 
        ) -> np.ndarray:
    """
    Procedure to get 
    
    Params:
    ----------
    segmentation:
        np.ndarray containing inferred organelle object

    florescence:
        np.ndarray containing raw organelle image

    organelle:
        channel of organelle to process: nuclei,  NUC_CH = 0, LYSO_CH = 1, MITO_CH = 2, GOLGI_CH = 3, PEROXI_CH = 4, ER_CH = 5, LIPID_CH = 6, RESIDUAL_CH = 7

    in_image:
        optional np.ndarray containing raw organelle image

    mask:
        optional mask (soma or cytosol)
    
    stats:
        optiohal Prior class
    
    export:
        boolean flag to export - default True
    """
    # CHOOSE which inferred organelle to 
    if organelle == NUC_CH: #0
        organelle_name = 'nucleus'
    elif organelle == LYSO_CH: #1 
        organelle_name = 'lysosome'
    elif organelle == MITO_CH: #2
        organelle_name = 'mitochondria'
    elif organelle == GOLGI_CH: #3
        organelle_name = 'golgi'
    elif organelle == PEROXI_CH: #4
        organelle_name = 'peroxisome'
    elif organelle == ER_CH: #5
        organelle_name = 'endoplasmic reticulum'
    elif organelle == LIPID_CH: #6 
        organelle_name = 'lipid'
    elif organelle == RESIDUAL_CH: #7
        organelle_name = 'residual'

    print(organelle_name)

    target = segmentation[organelle].squeeze()
    target = apply_mask(target, mask.squeeze())    
    target_intensity = florescence[organelle].squeeze()

    summary_stats = collect_organelle_stats(
                                    target, 
                                    target_intensity
                                    )
    # test if it worked
    labels = label(img_in).astype("int").squeeze()

    # export

    # return


In [ ]:
properties = ['label']
extra_properties = []
size = True 
perimeter = True 
shape = True 
position = True 
moments = True

labels = label(img_in).astype("int").squeeze()
intensity_image = None if florescence is None else florescence.squeeze()
intensity = intensity_image is not None # depricate for now because the "infered object" doesn't have intensity


if size:
    properties = properties + ['area', 'bbox_area',  'equivalent_diameter'] #'convex_area',

if intensity:
    properties = properties + ['max_intensity', 'mean_intensity', 'min_intensity']
    # arguments must be in the specified order, matching regionprops
    def standard_deviation_intensity(region, intensities):
        return np.std(intensities[region])
    extra_properties.append(standard_deviation_intensity)

if perimeter:
    if len(labels.shape) == 2:
        properties = properties + ['perimeter', 'perimeter_crofton']
    else:
        print("Perimeter measurements are not supported in 3D")
        # warnings.warn("Perimeter measurements are not supported in 3D")
        properties = properties + ['perimeter', 'perimeter_crofton']

if shape:
    properties = properties + ['solidity', 'extent', 'feret_diameter_max', 'local_centroid']
    if len(labels.shape) == 2:
        properties = properties + ['major_axis_length', 'minor_axis_length', 'orientation', 'eccentricity']
        # we need these two to compute some shape descriptors
        if not size:
            properties = properties + ['area']
        if not perimeter:
            properties = properties + ['perimeter']
    else:
        properties = properties + ['moments_central']
    # euler_number,

if position:
    properties = properties + ['centroid', 'bbox', 'weighted_centroid']

if moments:
    properties = properties + ['moments', 'moments_normalized']
    if 'moments_central' not in properties:
        properties = properties + ['moments_central']
    if len(labels.shape) == 2:
        properties = properties + ['moments_hu']

# todo:
# weighted_local_centroid
# weighted_moments
# weighted_moments_central
# weighted_moments_hu
# weighted_moments_normalized

# quantitative analysis using scikit-image's regionprops
print(labels.shape)
print(properties)
table = regionprops_table(labels, intensity_image=intensity_image,
                            properties=properties, extra_properties=extra_properties)

In [ ]:
organelle = 1
mask = cytosol_mask
segmentation = np.stack([
                            nuclei_object,
                            lysosome_object,
                            mito_object,
                            golgi_object,
                            peroxi_object,
                            er_object,
                            lipid_object ])

florescence = img_2D[:-1] # throw out residual


florescence.shape, segmentation.shape, mask.shape


In [ ]:

# CHOOSE which inferred organelle to 
if organelle == NUC_CH: #0
    organelle_name = 'nucleus'
elif organelle == LYSO_CH: #1 
    organelle_name = 'lysosome'
elif organelle == MITO_CH: #2
    organelle_name = 'mitochondria'
elif organelle == GOLGI_CH: #3
    organelle_name = 'golgi'
elif organelle == PEROXI_CH: #4
    organelle_name = 'peroxisome'
elif organelle == ER_CH: #5
    organelle_name = 'endoplasmic reticulum'
elif organelle == LIPID_CH: #6 
    organelle_name = 'lipid'
elif organelle == RESIDUAL_CH: #7
    organelle_name = 'residual'

print(organelle_name)

target = segmentation[organelle].squeeze()
target = apply_mask(target, mask.squeeze())    
target_intensity = florescence[organelle].squeeze()

labels = label(target).astype("int").squeeze()


summary_stats = collect_organelle_stats(
                                target, 
                                target_intensity
                                )
# target.shape, target_intensity.shape, labels.shape

labels.max()



In [ ]:

# loop over all labeled organelles
for org_i in range(1,labels.max()+1):
    # extract org_i mask
    msk_i = labels == org_i
    intersect = 


In [ ]:
from skimage.measure import regionprops

label_image = labels

input_props = regionprops(
    label_image, intensity_image=None, cache=True
)


In [ ]:

input_props[0].coords.shape

In [ ]:

input_centroids = [np.int_(obj["centroid"]) for obj in input_props]
input_centroids[0]


In [ ]:

output_segmented = np.zeros_like(label_image)

for ind, arr in enumerate(input_centroids):
    output_segmented[tuple(arr)] = ind + 1
    break



In [ ]:
ind

In [ ]:
napari.view_image(output_segmented)

In [ ]:

summary_stats = collect_organelle_stats(
                                target, 
                                target_intensity
                                )


import pandas as pd

stats_table = pd.DataFrame(summary_stats)
stats_table.head()

In [ ]:
# np.stack(img_layers, axis=0).shape
nuclei_table =  collect_organelle_stats(nuclei_object)


import pandas as pd

nuc = pd.DataFrame(nuclei_table)


In [ ]:
nuc

In [ ]:
lysosome_table =  collect_organelle_stats(lysosome_object)


In [ ]:
napari.view_image(lysosome_object)

In [ ]:
napari.view_labels(label(lysosome_object))

In [ ]:
lysosome_table

## Visualize with `napari` 1
Visualize the first-pass segmentation and labeling with `napari`.

In [ ]:
viewer = napari.Viewer()

In [ ]:
viewer.add_image(
    nuclei_object,
    scale=scale,
    colormap='blue', 
    blending='additive'
)
viewer.add_image(
    lysosome_object,
    scale=scale,
    colormap='cyan', 
    blending='additive'
)

viewer.add_image(
    mito_object,
    scale=scale,
    colormap='green', 
    blending='additive'
)

viewer.add_image(
    golgi_object,
    scale=scale,
    colormap='yellow', 
    blending='additive'
)


viewer.add_image(
    peroxi_object,
    scale=scale,
    colormap='bop orange', 
    blending='additive'
)


viewer.add_image(
    er_object,
    scale=scale,
    blending='additive')

viewer.add_image(
    lipid_object,
    scale=scale,
    blending='additive')



In [ ]:
viewer.scale_bar.visible = True
from napari.utils.notebook_display import nbscreenshot

# viewer.dims.ndisplay = 3
# viewer.camera.angles = (-30, 25, 120)
nbscreenshot(viewer, canvas_only=True)

In [ ]:
viewer.close()


There may be a bug where the input images to the "infer_*" functions are modified in place and we might need to access them.  _MASKING_ seems to be the problem.  Also need to be clear about _when_ to apply the mask.

In [ ]:
from infer_subc_2d.utils.img import select_z_from_raw
###########
# infer organelles
##########
def _fixed_infer_organelles(img_data):
    """
    wrapper to infer all organelles from a single multi-channel image
    """
    # ch_to_agg = (LYSO_CH, MITO_CH, GOLGI_CH, PEROXI_CH, ER_CH, LIPID_CH)

    # nuc_ch = NUC_CH
    # optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg)
    # # Stage 1:  nuclei, soma, cytosol
    # img_2D = select_z_from_raw(img_data, optimal_Z)
    img_2D = fixed_get_optimal_Z_image(img_data)

    soma_mask = fixed_infer_soma(img_2D)

    nuclei_object = fixed_infer_nuclei(img_2D, soma_mask)

    cytosol_mask = infer_cytosol(nuclei_object, soma_mask)

    # cyto masked objects.
    lysosome_object = fixed_infer_lysosome(img_2D, cytosol_mask)
    mito_object = fixed_infer_mitochondria(img_2D, cytosol_mask)
    golgi_object = fixed_infer_golgi(img_2D, cytosol_mask)
    peroxi_object = fixed_infer_peroxisome(img_2D, cytosol_mask)
    er_object = fixed_infer_endoplasmic_reticulum(img_2D, cytosol_mask)
    lipid_object = fixed_infer_lipid(img_2D, cytosol_mask)

    img_layers = [
        nuclei_object,
        lysosome_object,
        mito_object,
        golgi_object,
        peroxi_object,
        er_object,
        lipid_object,
        soma_mask,
        cytosol_mask,
    ]

    layer_names = [
        "nuclei",
        "lysosome",
        "mitochondria",
        "golgi",
        "peroxisome",
        "er",
        "lipid_body",
        "soma_mask",
        "cytosol_mask",
    ]
    # TODO: pack outputs into something napari readable
    img_out = np.stack(img_layers, axis=0)
    return (img_out, layer_names, optimal_Z)


In [ ]:

def stack_organelle_objects(soma_mask,
                            nuclei_object,
                            cytosol_mask,
                            lysosome_object,
                            mito_object,
                            golgi_object,
                            peroxi_object,
                            er_object,
                            lipid_object) -> np.ndarray:
    """ wrapper to stack the inferred objects into a single numpy.ndimage """
    img_layers = [soma_mask,
                            nuclei_object,
                            cytosol_mask,
                            lysosome_object,
                            mito_object,
                            golgi_object,
                            peroxi_object,
                            er_object,
                            lipid_object]
    return np.stack(img_layers, axis=0) 

In [ ]:
def stack_organelle_layers(*layers) -> np.ndarray:
    """ wrapper to stack the inferred objects into a single numpy.ndimage """

    return np.stack(layers, axis=0) 

In [ ]:
stacked_organelle_objects(*img_layers).shape

In [ ]:

inferred_organelles, layer_names, optimal_Z = _fixed_infer_organelles(img_data)


In [ ]:
viewer = napari.Viewer()


In [ ]:
cmaps = ['blue','cyan','green','yellow','bop orange','magenta','gray','gray','gray']

for i,organelle in enumerate(inferred_organelles):
    viewer.add_image(
        organelle,
        scale=scale,
        blending='additive',
        name = layer_names[i],
        colormap=cmaps[i]
    )


In [ ]:
viewer.scale_bar.visible = True

nbscreenshot(viewer, canvas_only=True)

In [ ]:
meta_dict

In [ ]:

##################
# export
##################

def _export_infer_organelles(img_out, layer_names, meta_dict, data_root_path):
       # get some top-level info about the RAW data
    # channel_names = meta_dict['name']
    # img = meta_dict['metadata']['aicsimage']
    # scale = meta_dict['scale']
    # channel_axis = meta_dict['channel_axis']
    img_name = meta_dict['file_name']
    # add params to metadata
    meta_dict['layer_names'] = layer_names
    out_path = data_root_path / "inferred_objects" 
    img_name_out = 'binarized_' + img_name.split("/")[-1].split(".")[0]

    out_file_n = export_ome_tiff(img_out, meta_dict, img_name_out, str(out_path)+"/", layer_names)
    print(f"saved file: {out_file_n}")
    return out_file_n

out_file_n = _export_infer_organelles(inferred_organelles, layer_names, meta_dict, data_root_path)



In [ ]:
out_file_n

----------------------

run a batch of ALL the images

First get all the images

In [ ]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"



now build a function to loop over them all and export




In [ ]:

def batch_process_all_czi(data_root_path):

    # linearly unmixed ".czi" files are here
    data_path = data_root_path / "raw"
    im_type = ".czi"
    # get the list of all files
    img_file_list = list_image_files(data_path,im_type)
    files_generated = []
    for czi_file in img_file_list:
        out_fn = process_czi_image(czi_file)
        files_generated.append(out_fn)

    print(f"generated {len(files_generated)} ")
    return files_generated

def process_czi_image(czi_file_name):
    """wrapper for processing"""

    img_data,meta_dict = read_czi_image(czi_file_name)
    # # get some top-level info about the RAW data
    # channel_names = meta_dict['name']
    # img = meta_dict['metadata']['aicsimage']
    # scale = meta_dict['scale']
    # channel_axis = meta_dict['channel_axis']

    inferred_organelles, layer_names,optimal_Z = _infer_organelles(img_data)
    meta_dict['z_slice'] = optimal_Z
    out_file_n = _export_infer_organelles(inferred_organelles, layer_names, meta_dict, data_root_path)

    ## TODO:  collect stats... 

    return out_file_n

# chan_name = 'nuclei'
# out_path = data_root_path / "inferred_objects" 
# object_name = 'NU_object'

# NU_bioim = read_input_image( out_path/ f"{object_name}.ome.tiff"  )
# NU_object = NU_bioim.image
# NU_labels = label(NU_object)

# # calculate a filter dimension for median filtering which considers the difference in scale of Z
# z_factor = scale[0]//scale[1]
# med_filter_size = 4 #2D 
# med_filter_size_3D = (1,med_filter_size,med_filter_size)  # set the scale for a typical median filter

In [ ]:
output_tiffs = batch_process_all_czi(data_root_path)


Write the `infer_soma` spec to the widget json

In [ ]:
from infer_subc_2d.organelles_config.helper import add_function_spec_to_widget_json

_fixed_infer_organelles =  {
        "name": "infer all organelles (fixed parameters)",
        "python::module": "infer_subc_2d.batch.batch_process",
        "python::function": "fixed_infer_organelles",
        "parameters": None
        }

add_function_spec_to_widget_json("fixed_infer_organelles",_fixed_infer_organelles)

In [ ]:

_stack_organelle_objects =  {
        "name": "stack organelles, argv spelled out",
        "python::module": "infer_subc_2d.batch",
        "python::function": "stack_organelle_objects",
        "parameters": None
        }

add_function_spec_to_widget_json("stack_organelle_objects",_stack_organelle_objects)


_stack_organelle_layers =  {
        "name": "stack organelles, *argv ",
        "python::module": "infer_subc_2d.batch",
        "python::function": "stack_organelle_layers",
        "parameters": None
        }

add_function_spec_to_widget_json("stack_organelle_layers",_stack_organelle_layers)


-------------------------------
## Write workflow .json
Now that we've added our function specs we can compose workflows.

In [ ]:

def make_fixed_infer_organelles_batch_dict():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # Stage 1:  nuclei, soma, cytosol
    ###################   
    step_name.append("1")
    function_name.append("fixed_get_optimal_Z_img")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)

    step_name.append("2")
    function_name.append("fixed_infer_soma")
    category.append("core")
    parameter_values.append( None )
    parent.append(1)

    step_name.append("3")
    function_name.append("fixed_infer_nuclei")
    category.append("core")
    parameter_values.append( None )
    parent.append([1,2])

    step_name.append("4")
    function_name.append("infer_cytosol")
    category.append("core")
    parameter_values.append(dict(erode_nuclei = True ))
    parent.append([2,3])

    ###################
    # Stage 2:  cyto masked objects
    ###################   
    step_name.append("5")
    function_name.append("fixed_infer_lysosome")
    category.append("core")
    parameter_values.append(None )
    parent.append([1,4])

    
    step_name.append("6")
    function_name.append("fixed_infer_mitochondria")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("7")
    function_name.append("fixed_infer_golgi")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("8")
    function_name.append("fixed_infer_peroxisome")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("9")
    function_name.append("fixed_infer_er") 
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("10")
    function_name.append("fixed_infer_lipid") 
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("11")
    function_name.append("stack_organelle_objects") 
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([2,3,4,5,6,7,8,9,10])


    # TODO: add export functions

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
    
    return out_dict



In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json

infer_fixed_infer_organelles_batch_dict = make_fixed_infer_organelles_batch_dict()

write_workflow_json("infer_fixed_infer_organelles_batch", infer_fixed_infer_organelles_batch_dict)

In [ ]:

def make_fixed_infer_organelles_batch_dict2():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw
    """
    
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # Stage 1:  nuclei, soma, cytosol
    ###################   
    step_name.append("1")
    function_name.append("fixed_get_optimal_Z_img")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)

    step_name.append("2")
    function_name.append("fixed_infer_soma")
    category.append("core")
    parameter_values.append( None )
    parent.append(1)

    step_name.append("3")
    function_name.append("fixed_infer_nuclei")
    category.append("core")
    parameter_values.append( None )
    parent.append([1,2])

    step_name.append("4")
    function_name.append("infer_cytosol")
    category.append("core")
    parameter_values.append(dict(erode_nuclei = True ))
    parent.append([2,3])

    ###################
    # Stage 2:  cyto masked objects
    ###################   
    step_name.append("5")
    function_name.append("fixed_infer_lysosome")
    category.append("core")
    parameter_values.append(None )
    parent.append([1,4])

    
    step_name.append("6")
    function_name.append("fixed_infer_mitochondria")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("7")
    function_name.append("fixed_infer_golgi")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("8")
    function_name.append("fixed_infer_peroxisome")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("9")
    function_name.append("fixed_infer_er") 
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("10")
    function_name.append("fixed_infer_lipid") 
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])


    step_name.append("11")
    function_name.append("stack_organelle_objects") 
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([2,3,4,5,6,7,8,9,10])

    # TODO: add export functions

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
    
    return out_dict



In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json

infer_fixed_infer_organelles_batch_dict2 = make_fixed_infer_organelles_batch_dict2()

write_workflow_json("infer_fixed_infer_organelles_batch2", infer_fixed_infer_organelles_batch_dict2)

In [ ]:
#TODO: make infer_organelles function with the exhaustive list of parmaters
_infer_organelles =  {
        "name": "Infer Endoplasmic Reticulum",
        "python::module": "infer_subc_2d.organelles",
        "python::function": "infer_endoplasmic_reticulum",
        "parameters": {
                "filament_scale": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": 10,
                        "min": 0,
                        "widget_type": "slider"
                },
                "filament_cut": {
                        "data_type": "float",
                        "increment": 0.001,
                        "max": 0.5,
                        "min": 0,
                        "widget_type": "slider"
                },
                "small_obj_w": {
                        "data_type": "int",
                        "increment": 1,
                        "max": 50,
                        "min": 1,
                        "widget_type": "slider"
                }
        }
}

add_function_spec_to_widget_json("eeeinfer_organelles", _infer_organelles, overwrite=True )



35 files processed in 6 minutes 47 seconds!!

In [ ]:

tiff_img_data,tiff_meta_dict = read_czi_image(output_tiffs[-1])


In [ ]:
tiff_img_data.shape

img = tiff_meta_dict['metadata']['aicsimage']
img.dims

In [ ]:
from aicsimageio.writers import OmeTiffWriter
data_in = img_out
channel_names = [layer_names]
image_names = [img_name]
print(image_names)
# chan_names = meta_in['metadata']['aicsimage'].channel_names
dimension_order = ["CZYX"]

num_images = len(  [data_in.shape])
if data_in.dtype == "bool":
    data_in = data_in.astype(np.uint8)
    data_in[data_in > 0] = 255

physical_pixel_sizes = [meta_dict["metadata"]["aicsimage"].physical_pixel_sizes]
out_ome = OmeTiffWriter.build_ome(
        [data_in.shape],
        [data_in.dtype],
        channel_names=channel_names,  # type: ignore
        image_name=image_names,
        physical_pixel_sizes=physical_pixel_sizes,
        dimension_order=dimension_order,
    )